In [1]:
import pandas as pd
import numpy as np

In [2]:
DiabetesAllDummy = pd.read_csv('DiabetesAllDummy.csv', index_col=0)
#DiabetesOrdMed = pd.read_csv('DiabetesOrdMed.csv', index_col=0)

In [3]:
#Set a random index of 20% of the values in our dataset. This is the test dataset and will *only* be used for testing our models (sparingly!)
np.random.seed(100)
length = len(DiabetesAllDummy)
testIdx = np.random.choice(range(length), size=int(round(0.2*length)), replace=False)
trainIdx = list(set(range(length))-set(testIdx))

In [4]:
#Let's just start by looking at a simple linear regression:
#Yes, we have a categorical variable as our output. This will not be valid, but will show right away variables which could be important:

DiabetesTrain = DiabetesAllDummy[DiabetesAllDummy['IsTrain']==1]
DiabetesTrain.index = list(range(len(DiabetesTrain)))

DiabetesTest = DiabetesAllDummy[DiabetesAllDummy['IsTrain']==0]
DiabetesTest.index = list(range(len(DiabetesTest)))

In [5]:
#Try simple K-fold (5) random forest classifier using default conditions:

from sklearn.ensemble import RandomForestClassifier as rfc

rfc = rfc()

import sklearn.model_selection as ms
ms_k5 = ms.KFold(n_splits=5, shuffle=True)

#First try with all features:

#Create X and Y variables (replace 2 to 1 and 1 to 0)
trainX = DiabetesTrain.drop('readmitted', axis=1)
trainY = DiabetesTrain['readmitted'].replace([2, 1], [1, 0])

testX = DiabetesTest.drop('readmitted', axis=1)
testY = DiabetesTest['readmitted'].replace([2, 1], [1, 0])


#Instantiate the K-fold generator object:
np.random.seed(0)
DiabetesAD5Fold = ms_k5.split(trainX, trainY)

#Produce list of test R^2 scores, and Actual vs Predicted lists for the individual runs
ScoreList = []
PredictList = []
PureTestScore = []
TrainScores = []
ActualValues = []

for train, test in DiabetesAD5Fold:

    
    #Run the fit using the train data for each K
    rfc.fit(trainX.iloc[train,], trainY[train])
    #Run your predicion for the "missing" K-part
    p = rfc.predict(trainX.iloc[test,])
    actual = trainY[train].values
    #Check your schore for the missing K-part
    R2 = rfc.score(trainX.iloc[test,], trainY[test])
    #Run a test on the completely untouched test 20%
    TestR2 = rfc.score(testX, testY)
    TrainScore = rfc.score(trainX.iloc[train,], trainY[train])
    
    #Append these scores to the lists above
    ScoreList.append(R2)
    PureTestScore.append(TestR2)
    PredictList.append(p)
    TrainScores.append(TrainScore)
    ActualValues.append(actual)
    
    #Make predictions for the completely untouched 20%
    PredictionsTest = rfc.predict(testX)
    
    #Use these predictions to calculate RMSLE for the untouched 20% and append
    #RMSLEvalue = np.sqrt(np.mean(np.power(np.log1p(testY)-np.log1p(PredictionsTest), 2)))
    #RMSLE.append(RMSLEvalue)

    


In [6]:
#These are the internal train scores, based on the logistic regression
TrainScores

[0.9803486974573493,
 0.9801926108605054,
 0.9794277865359702,
 0.9797555683893424,
 0.9798027096210277]

In [7]:
#These are the K-test scores, based on the logistic regression
ScoreList

[0.883373915215084,
 0.8875569707186115,
 0.8841231191858651,
 0.8844352875070238,
 0.885551948051948]

In [8]:
#These are the scores for each of the 5 ensemble models, compared to the untouched 20% test index
PureTestScore

[0.8849725411882177,
 0.8853220169745382,
 0.8845232151772342,
 0.8845731402895657,
 0.8846729905142287]

In [9]:
#These are the prediction lists from the train, compared to actual Y values for the train:
PredictList

[array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64)]

In [10]:
#These values may sound impressive, but they absolutely aren't. 11.1 percent of patients return under 30 days, so simply guessing "No" will get you 88.8% accuracy:
1-np.sum(trainY)/len(trainY)

0.8865441286649018

In [73]:
TrainScores

[0.886540652754148,
 0.8857133937908752,
 0.8867123480106763,
 0.8863533488379353,
 0.8863239058500343]

In [74]:
ScoreList

[0.8856215271274271,
 0.8886183430105513,
 0.8848098894924143,
 0.8861834301055129,
 0.8861138861138861]

In [75]:
PureTestScore

[0.8865202196704942,
 0.8861208187718422,
 0.8861707438841737,
 0.8863205192211683,
 0.8864203694458312]

In [76]:
#The accuracy with reduced features was no better than the overall accuracy with all features. Let's check what might be going on:

In [11]:
np.sum(PredictList[0])

78

In [15]:
np.sum(trainY)

9086

In [12]:
len(PredictList[0])

16017

In [14]:
'''This seems to be the biggest problem. Out of ~16,000 observations (in the first K group, as an example), only 78 of them
were predicted to return within 30 days. We know that 11% should, or rougly 1,800. We need to tune this model to be more
generous in predicting a positive outcome'''

'This seems to be the biggest problem. Out of ~16,000 observations (in the first K group, as an example), only 78 of them\nwere predicted to return within 30 days. We know that 11% should, or rougly 1,800. We need to tune this model to be more\ngenerous in predicting a positive outcome'

In [80]:
from sklearn.model_selection import RandomizedSearchCV

#We will perform a grid search to find the optimal hyperparameters for our RF algorithm and test using our K-fold data:

# C value is the one most important for tuning a logistic regression. Let's see how varying this value affects the score:
C_range = [0.001, 0.01, 0.1, 1, 10, 100, 100]


# Create the random grid
random_grid = {'C': C_range}

# Use the random grid to search for best C hyperparameter:

# First create the base model to tune
lgrCV = lgr()

# Random search of parameters, using 5-fold cross validation, 
lgr_random = RandomizedSearchCV(estimator = lgrCV, param_distributions = random_grid, n_iter = 7, cv = 5, verbose=2, random_state=42, n_jobs = 2)

# Fit the random search model
lgr_random.fit(trainX, trainY)

#Then print the best parameters using best_params_
lgr_random.best_params_

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=2)]: Done  35 out of  35 | elapsed:   27.7s finished


{'C': 0.001}

In [81]:
#Let's try the logistic regression with c=0.001 and see how that works:

from sklearn.linear_model import LogisticRegression as lgr

lgr = lgr()
lgr.set_params(C=0.001)

import sklearn.model_selection as ms
ms_k5 = ms.KFold(n_splits=5, shuffle=True)

#First try with all features:

#Create X and Y variables (replace 2 to 1 and 1 to 0)
trainX = DiabetesTrain.drop('readmitted', axis=1)
trainY = DiabetesTrain['readmitted'].replace([2, 1], [1, 0])

testX = DiabetesTest.drop('readmitted', axis=1)
testY = DiabetesTest['readmitted'].replace([2, 1], [1, 0])


#Instantiate the K-fold generator object:
np.random.seed(0)
DiabetesAD5Fold = ms_k5.split(trainX, trainY)

#Produce list of test R^2 scores, and Actual vs Predicted lists for the individual runs
ScoreList = []
PredictList = []
PureTestScore = []
TrainScores = []
ActualValues = []

for train, test in DiabetesAD5Fold:

    
    #Run the fit using the train data for each K
    lgr.fit(trainX.iloc[train,], trainY[train])
    #Run your predicion for the "missing" K-part
    p = lgr.predict(trainX.iloc[test,])
    actual = trainY[train].values
    #Check your schore for the missing K-part
    R2 = lgr.score(trainX.iloc[test,], trainY[test])
    #Run a test on the completely untouched test 20%
    TestR2 = lgr.score(testX, testY)
    TrainScore = lgr.score(trainX.iloc[train,], trainY[train])
    
    #Append these scores to the lists above
    ScoreList.append(R2)
    PureTestScore.append(TestR2)
    PredictList.append(p)
    TrainScores.append(TrainScore)
    ActualValues.append(actual)
    
    #Make predictions for the completely untouched 20%
    PredictionsTest = lgr.predict(testX)
    
    #Use these predictions to calculate RMSLE for the untouched 20% and append
    #RMSLEvalue = np.sqrt(np.mean(np.power(np.log1p(testY)-np.log1p(PredictionsTest), 2)))
    #RMSLE.append(RMSLEvalue)

In [82]:
PureTestScore

[0.8860708936595107,
 0.8857713429855217,
 0.8858711932101847,
 0.8858212680978532,
 0.8858212680978532]

In [83]:
#Either logistic regression does not work for this data, or something is going wrong here. The predicted scores are up by the slightest amount

In [84]:
#Finally, try the logistic regression with c=0.001 and reduced features and see how that works:

from sklearn.linear_model import LogisticRegression as lgr

lgr = lgr()
lgr.set_params(C=0.001)

import sklearn.model_selection as ms
ms_k5 = ms.KFold(n_splits=5, shuffle=True)

#First try with all features:



#Create X and Y variables (replace 2 to 1 and 1 to 0)
trainX = DiabetesTrainM.drop('readmitted', axis=1)
trainY = DiabetesTrainM['readmitted'].replace([2, 1], [1, 0])

testX = DiabetesTestM.drop('readmitted', axis=1)
testY = DiabetesTestM['readmitted'].replace([2, 1], [1, 0])


#Instantiate the K-fold generator object:
np.random.seed(0)
DiabetesAD5Fold = ms_k5.split(trainX, trainY)

#Produce list of test R^2 scores, and Actual vs Predicted lists for the individual runs
ScoreList = []
PredictList = []
PureTestScore = []
TrainScores = []
ActualValues = []

for train, test in DiabetesAD5Fold:

    
    #Run the fit using the train data for each K
    lgr.fit(trainX.iloc[train,], trainY[train])
    #Run your predicion for the "missing" K-part
    p = lgr.predict(trainX.iloc[test,])
    actual = trainY[train].values
    #Check your schore for the missing K-part
    R2 = lgr.score(trainX.iloc[test,], trainY[test])
    #Run a test on the completely untouched test 20%
    TestR2 = lgr.score(testX, testY)
    TrainScore = lgr.score(trainX.iloc[train,], trainY[train])
    
    #Append these scores to the lists above
    ScoreList.append(R2)
    PureTestScore.append(TestR2)
    PredictList.append(p)
    TrainScores.append(TrainScore)
    ActualValues.append(actual)
    
    #Make predictions for the completely untouched 20%
    PredictionsTest = lgr.predict(testX)
    
    #Use these predictions to calculate RMSLE for the untouched 20% and append
    #RMSLEvalue = np.sqrt(np.mean(np.power(np.log1p(testY)-np.log1p(PredictionsTest), 2)))
    #RMSLE.append(RMSLEvalue)

In [85]:
PureTestScore

[0.8863704443334998,
 0.8865202196704942,
 0.8865202196704942,
 0.8865701447828258,
 0.8865701447828258]

In [86]:
#Finally, try the logistic regression with c=0.001 and reduced features and see how that works:

from sklearn.linear_model import LogisticRegression as lgr

lgr = lgr()
lgr.set_params(C=1000)

import sklearn.model_selection as ms
ms_k5 = ms.KFold(n_splits=5, shuffle=True)

#First try with all features:



#Create X and Y variables (replace 2 to 1 and 1 to 0)
trainX = DiabetesTrainM.drop('readmitted', axis=1)
trainY = DiabetesTrainM['readmitted'].replace([2, 1], [1, 0])

testX = DiabetesTestM.drop('readmitted', axis=1)
testY = DiabetesTestM['readmitted'].replace([2, 1], [1, 0])


#Instantiate the K-fold generator object:
np.random.seed(0)
DiabetesAD5Fold = ms_k5.split(trainX, trainY)

#Produce list of test R^2 scores, and Actual vs Predicted lists for the individual runs
ScoreList = []
PredictList = []
PureTestScore = []
TrainScores = []
ActualValues = []

for train, test in DiabetesAD5Fold:

    
    #Run the fit using the train data for each K
    lgr.fit(trainX.iloc[train,], trainY[train])
    #Run your predicion for the "missing" K-part
    p = lgr.predict(trainX.iloc[test,])
    actual = trainY[train].values
    #Check your schore for the missing K-part
    R2 = lgr.score(trainX.iloc[test,], trainY[test])
    #Run a test on the completely untouched test 20%
    TestR2 = lgr.score(testX, testY)
    TrainScore = lgr.score(trainX.iloc[train,], trainY[train])
    
    #Append these scores to the lists above
    ScoreList.append(R2)
    PureTestScore.append(TestR2)
    PredictList.append(p)
    TrainScores.append(TrainScore)
    ActualValues.append(actual)
    
    #Make predictions for the completely untouched 20%
    PredictionsTest = lgr.predict(testX)
    
    #Use these predictions to calculate RMSLE for the untouched 20% and append
    #RMSLEvalue = np.sqrt(np.mean(np.power(np.log1p(testY)-np.log1p(PredictionsTest), 2)))
    #RMSLE.append(RMSLEvalue)

In [87]:
PureTestScore

[0.8865202196704942,
 0.8862206689965052,
 0.8861707438841737,
 0.8863205192211683,
 0.8863205192211683]

In [88]:
np.sum(PredictList[0])

54